In [1]:
import numpy as np
import numba
import matplotlib.pyplot as plt
import sympy as sym
import scipy.optimize as sopt
#plt.style.use('presentation')
%matplotlib notebook

def d2np(d):
    
    names = []
    numbers = ()
    dtypes = []
    for item in d:
        names += item   
        if type(d[item]) == float:
            numbers += (d[item],)
            dtypes += [(item,float)]
        if type(d[item]) == int:
            numbers += (d[item],)
            dtypes += [(item,int)]
        if type(d[item]) == np.ndarray:
            numbers += (d[item],)
            dtypes += [(item,np.float64,d[item].shape)]
    return np.array([numbers],dtype=dtypes)

### PSAT model

In [15]:
X1d,X1q,X_l,i_d,i_q,v_d,v_q,e1d,e1q,R_s = sym.symbols('X1d,X1q,X_l,i_d,i_q,v_d,v_q,e1d,e1q,R_s')  


eq_id = v_q + R_s*i_q - e1q + (X1d - X_l)*i_d
eq_iq = v_d + R_s*i_d - e1d - (X1q - X_l)*i_q

s = sym.solve([eq_id,eq_iq],[i_d,i_q])

for item in s:
    print(item, '=', s[item])




i_d = (R_s*(e1d - v_d) + (X1q - X_l)*(e1q - v_q))/(R_s**2 + (X1d - X_l)*(X1q - X_l))
i_q = (R_s*(e1q - v_q) - (X1d - X_l)*(e1d - v_d))/(R_s**2 + (X1d - X_l)*(X1q - X_l))


In [49]:
d =dict(
H = 0.7,
X_d = 2.06,
X_q = 2.5,
X1d = 0.398,
X1q = 0.3,
X2d = 0.254,
X2q = 0.254,
X_l = 0.1,
R_s = 0.05,
D = 0.01,
Omega_b = 2*np.pi*50,
T1d0 = 7.8,
T1q0 = 3.0,
T2d0 = 0.066,
T2q0 = 0.075,
V_m = 1.0,
theta= 0.0,
delta = 0.2,
omega = 1.0,
e_fd = 1.1,
p_m = 1.0,
p_e = 0.0,
i_d = 0.0,
i_q = 0.0,
x=np.zeros((4,1)),
f=np.zeros((4,1)),
y=np.zeros((2,1)),
g=np.zeros((2,1))
)

S_n = 5.0e6
U_n = 400.0
X_d = 1.81
X_q = 1.76
X1d = 0.3
X1q = 0.65
X2d = 0.23
X2q = 0.25
X_l = 0.15
R_s = 0.003
V_dc_n = 5e3

d =dict(
S_n = S_n,
U_n = U_n,
V_dc_n = V_dc_n,
H = 3.5,
X_d = X_d,
X_q = X_q,
X1d = X1d,
X1q = X1q,
X2d = X2d,
X2q =X2q,
X_l = X_l,
R_s = R_s,
D = 0.01,
Omega_b = 2*np.pi*50,
T1d0 = 8.0,
T1q0 = 1.0,
T2d0 = 0.03,
T2q0 = 0.07,
V_m = 1.0,
theta= 0.0,
delta = 0.2,
omega = 1.0,
e1q = 0.2,
e1d = 1.0,
e_fd = 1.1,
p_m = 1.0,
p_e = 0.0,
P = 1.0,
Q = 0.0,
P_pu = 1.0,
Q_pu = 0.0,
i_d = 0.0,
i_q = 0.0,
I_m = 0.0,
theta_i = 0.0,
x=np.zeros((4,1)),
f=np.zeros((4,1)),
y=np.zeros((2,1)),
g=np.zeros((2,1))
)


struct = d2np(d)

for item in d:
    print("    {:s} = struct[i]['{:s}']".format(item,item) )

    S_n = struct[i]['S_n']
    U_n = struct[i]['U_n']
    V_dc_n = struct[i]['V_dc_n']
    H = struct[i]['H']
    X_d = struct[i]['X_d']
    X_q = struct[i]['X_q']
    X1d = struct[i]['X1d']
    X1q = struct[i]['X1q']
    X2d = struct[i]['X2d']
    X2q = struct[i]['X2q']
    X_l = struct[i]['X_l']
    R_s = struct[i]['R_s']
    D = struct[i]['D']
    Omega_b = struct[i]['Omega_b']
    T1d0 = struct[i]['T1d0']
    T1q0 = struct[i]['T1q0']
    T2d0 = struct[i]['T2d0']
    T2q0 = struct[i]['T2q0']
    V_m = struct[i]['V_m']
    theta = struct[i]['theta']
    delta = struct[i]['delta']
    omega = struct[i]['omega']
    e1q = struct[i]['e1q']
    e1d = struct[i]['e1d']
    e_fd = struct[i]['e_fd']
    p_m = struct[i]['p_m']
    p_e = struct[i]['p_e']
    P = struct[i]['P']
    Q = struct[i]['Q']
    P_pu = struct[i]['P_pu']
    Q_pu = struct[i]['Q_pu']
    i_d = struct[i]['i_d']
    i_q = struct[i]['i_q']
    I_m = struct[i]['I_m']
    theta_i = struct[i]['theta_i']
    x = struct[i]['x']


In [60]:
Z_b = U_n**2/S_n

#Z_pos = (R_s + 0.5j*(X2d+X2q))*Z_b
Z_neg = (R_s + 0.5j*(X2d+X2q))*Z_b
Z_zero = (R_s + 0.5j*(X2d+X2q))*Z_b

In [61]:
alpha = np.exp(2.0/3*np.pi*1j)
A_0a =  np.array([[1, 1, 1],
                  [1, alpha**2, alpha],
                  [1, alpha, alpha**2]])

A_a0 = 1/3* np.array([[1, 1, 1],
                      [1, alpha, alpha**2],
                      [1, alpha**2, alpha]])

V_zero = 0.5*np.exp(1j*0.0)
V_neg  = 0.0*np.exp(1j*0.0)
V_pos  = 1.0*np.exp(1j*0.0)

V_012 = np.array([[V_zero],[V_pos],[V_neg]])

V_abc = A_0a @ V_012

In [62]:
Z_neg = (R_s + 0.5j*(X2d+X2q))*Z_b
Z_zero = (R_s + 0.5j*(X2d+X2q))*Z_b

In [63]:
V_abc_m = np.abs(V_abc)
np.sum(V_abc_m)/3

1.0773502691896257

In [64]:
U_abc= V_abc - V_abc[[1,2,0]]


In [150]:
i = 0
X_d = struct[i]['X_d']
X_q = struct[i]['X_q']
X_l = struct[i]['X_l']
R_s = struct[i]['R_s']

I_zero = 1000.0*np.exp(1j*0.0)
I_neg  = 0.0*np.exp(1j*0.0)
I_pos  = 1000.0*np.exp(1j*0.0)


I_012 = np.array([[I_zero],[I_pos],[I_neg]])


V_pos = 1.0*np.exp(1j*0.0)
V_012 = np.array([[0],[V_pos],[0]])

for it in range(10):
    I_abc = np.array([[-V_abc[0,0]/(0.02)],[0],[0]])
    #I_abc = -V_abc/0.1
    I_012 = A_a0 @ I_abc

    I_zero = I_012[0,0]
    I_pos  = I_012[1,0]
    I_neg  = I_012[2,0]

#V_pos = 400/np.sqrt(3)*np.exp(1j*0.0)
    V_zero = I_zero*Z_zero
    V_neg  = I_neg*Z_neg

#e_fd = v_q + R_s*i_q + (X_d+X_l)*i_d
#e_fq = v_d + R_s*i_d - (X_q+X_l)*i_q

#delta = np.atan2(e_fq,e_fd)
    V_012 = np.array([[V_zero],[V_pos],[V_neg]])

    V_abc = A_0a @ V_012

delta = np.angle(V_pos + (R_s + 1j*(X_q-X_l))*I_pos)

v_dq = V_pos*np.exp(-1j*(delta-np.pi/2)) 
i_dq = I_pos*np.exp(-1j*(delta-np.pi/2))
v_d,v_q = v_dq.real,v_dq.imag
i_d,i_q = i_dq.real,i_dq.imag
    
e1q = v_q + R_s*i_q + (X1d - X_l)*i_d
e1d = v_d + R_s*i_d - (X1q - X_l)*i_q

e_fd = e1q + (X_d-X1d)*i_d 

abs(V_abc)

array([[ 0.96585844],
       [ 1.03895094],
       [ 1.03895094]])

In [151]:
I_pos

(-15.597815266992372+3.9803062422494895j)

In [149]:
e_fd

16.628289022423125

In [130]:
X_d

1.8100000000000001

In [122]:
e1q

508.66499389068582

In [21]:
V_zero = 0.0*np.exp(1j*0.0)
V_neg =  0.0*np.exp(1j*0.0)
V_pos = 1.0*np.exp(1j*0.0)

V_012 = np.array([[V_zero],[V_pos],[V_neg]])

I_pos = 1.0 
e_fd = 3.0
for it in range(5):
    delta = np.angle(V_pos + (R_s + 1j*(X_q-X_l))*I_pos)
    v_dq = V_pos*np.exp(-1j*(delta-np.pi/2)) 
    i_dq = I_pos*np.exp(-1j*(delta-np.pi/2))
    v_d,v_q = v_dq.real,v_dq.imag
    i_d,i_q = i_dq.real,i_dq.imag
    e1q = -(X_d-X1d)*i_d + e_fd
    e1d =  (X_q-X1q)*i_q
    i_d_ref = (R_s*(e1d - v_d) + (X1q - X_l)*(e1q - v_q))/(R_s**2 + (X1d - X_l)*(X1q - X_l))
    i_q_ref = (R_s*(e1q - v_q) - (X1d - X_l)*(e1d - v_d))/(R_s**2 + (X1d - X_l)*(X1q - X_l))
    i_d = i_d + 0.1*(i_d_ref - i_d)
    i_q = i_q + 0.1*(i_q_ref - i_q)
    i_dq = i_d + 1j*i_q
    I_pos = i_dq*np.exp(1j*(delta-np.pi/2))
    I_neg = V_neg/Z_neg
    I_zero = V_zero/Z_zero
    print(I_pos)

(1.60235918783-0.37026040282j)
(1.53809364591-0.330894662272j)
(1.54494272329-0.335091674711j)
(1.54421269832-0.334644344656j)
(1.54429050876-0.334692023984j)


In [19]:
print(I_zero)
print(I_pos)
print(I_neg)

0j
(1.54429050876-0.334692023984j)
0j


Given voltages V_a,V_b,V_c and 

In [67]:

#@numba.jit(nopython=True, cache=True)
def sm_ord4(struct,i=0):
    x_idx = 0
    y_idx = 0
    
    H = struct[i]['H']
    X_d = struct[i]['X_d']
    X_q = struct[i]['X_q']
    X1d = struct[i]['X1d']
    X1q = struct[i]['X1q']
    X2d = struct[i]['X2d']
    X2q = struct[i]['X2q']
    X_l = struct[i]['X_l']
    R_s = struct[i]['R_s']
    D = struct[i]['D']
    Omega_b = struct[i]['Omega_b']
    T1d0 = struct[i]['T1d0']
    T1q0 = struct[i]['T1q0']
    T2d0 = struct[i]['T2d0']
    T2q0 = struct[i]['T2q0']
    V_m = struct[i]['V_m']
    theta = struct[i]['theta']
    delta = struct[i]['delta']
    e_fd = struct[i]['e_fd']
    p_m = struct[i]['p_m']
    p_e = struct[i]['p_e']
    i_d = struct[i]['i_d']
    i_q = struct[i]['i_q']
    omega = struct[i]['omega']
    x = struct[i]['x']
    f = struct[i]['f']
    y = struct[i]['y']
    g = struct[i]['g']

    delta =  struct[i]['x'][x_idx+0,0] %(2*np.pi)
    omega  = struct[i]['x'][x_idx+1,0]
    e1q =    struct[i]['x'][x_idx+2,0]
    e1d =    struct[i]['x'][x_idx+3,0]
    
    #i_d =   struct[i]['y'][y_idx+0,0]
    #i_q =   struct[i]['y'][y_idx+1,0]
   
    pi = np.pi
    sin = np.sin
    cos = np.cos 
    
    v_d = V_m*sin(delta-theta)
    v_q = V_m*cos(delta-theta)  

    i_d = (R_s*(e1d - v_d) + (X1q - X_l)*(e1q - v_q))/(R_s**2 + (X1d - X_l)*(X1q - X_l))
    i_q = (R_s*(e1q - v_q) - (X1d - X_l)*(e1d - v_d))/(R_s**2 + (X1d - X_l)*(X1q - X_l))
 
    p_e = (v_q + R_s * i_q) * i_q + (v_d + R_s*i_d) * i_d
    
    ddelta = Omega_b * (omega-1)
    domega = 1.0/(2*H)*(p_m - p_e - D*(omega-1))
    de1q = 1/T1d0*(-e1q - (X_d-X1d)*i_d + e_fd)
    de1d = 1/T1q0*(-e1d + (X_q-X1q)*i_q)

    eq_id = v_q + R_s*i_q - e1q + (X1d - X_l)*i_d
    eq_iq = v_d + R_s*i_d - e1d - (X1q - X_l)*i_q
    
    P = v_d*i_d + v_q*i_q
    Q = v_q*i_d - v_d*i_q
   
    struct[i]['f'][x_idx+0,0] = float(ddelta)
    struct[i]['f'][x_idx+1,0] = float(domega)
    struct[i]['f'][x_idx+2,0] = float(de1q)
    struct[i]['f'][x_idx+3,0] = float(de1d)
    
    #struct[i]['g'][y_idx+0,0] = float(eq_id)
    #struct[i]['g'][y_idx+1,0] = float(eq_iq)

    struct[i]['p_e'] = float(p_e)
    struct[i]['i_d'] = float(i_d)
    struct[i]['i_q'] = float(i_q)
    struct[i]['delta'] = float(delta)
    struct[i]['omega'] = float(omega)
    struct[i]['e1q'] = float(e1q)
    struct[i]['e1d'] = float(e1d)
    struct[i]['P'] = float(P)
    struct[i]['Q'] = float(Q)
    return 0



In [76]:
#class problem(object)
def problem(lam, i=0):
    #print(lam.shape)
    struct[i]['x'] = lam[0:4,:]
  #  struct[i]['y'] = lam[4:6,:]
    Phi = np.zeros((4,1))
    sm_ord4(struct,i=0)
    Phi[0:4,:] = struct[i]['f']
   # Phi[4:6,:] = struct[i]['g']   
    return Phi
    
sm_ord4(struct,i=0)
i=0
lam_0 = np.ones((4,1))

struct[i]['D'] = 0.0
struct[i]['R_s'] = 0.003

P = 0.8
Q = 0.436
S = P - 1j*Q 
V_m = 1.0
theta = np.deg2rad(28.34)
V = V_m*np.exp(1j*theta)

I = S/np.conj(V)
R_s = struct[i]['R_s']
X_d,X_q = struct[i]['X_d'],struct[i]['X_q']
X1d,X1q,X_l = struct[i]['X1d'],struct[i]['X1q'],struct[i]['X_l']

E = V + (R_s + 1j*(X_q-X_l))*I
delta = np.angle(E)
#delta = 1.199
v_dq = V*np.exp(-1j*(delta-np.pi/2)) 
i_dq = I*np.exp(-1j*(delta-np.pi/2))

v_d,v_q = v_dq.real,v_dq.imag
i_d,i_q = i_dq.real,i_dq.imag

e1q = v_q + R_s*i_q + (X1d-X_l)*i_d
e1d = v_d + R_s*i_d - (X1q-X_l)*i_q
e_fd = e1q + (X_d - X1d)*i_d
    
delta_0 = delta

lam_0[0] = delta
lam_0[1] = 1.0
lam_0[2] = e1q
lam_0[3] = e1d
#lam_0[4] = i_d
#lam_0[5] = i_q

struct[i]['p_m'] = P+R_s*(i_d**2+i_q**2)
struct[i]['V_m'] = V_m
struct[i]['theta'] = theta
struct[i]['e_fd'] = e_fd
solution = sopt.broyden2(problem,lam_0,  maxiter=1000)
solution
print('p_e = {:0.8f} pu'.format(struct[i]['p_e']))
print('i_d = {:0.3f} pu'.format(struct[i]['i_d']))
print('i_q = {:0.3f} pu'.format(struct[i]['i_q']))
print('delta = {:0.3f} rad / {:0.3f} deg'.format(struct[i]['delta'],np.rad2deg(struct[i]['delta'])))
print('omega = {:0.3f} pu'.format(struct[i]['omega']))
print('e1q = {:0.3f} pu'.format(struct[i]['e1q']))
print('e1d = {:0.3f} pu'.format(struct[i]['e1d']))
print('e1q0 = {:0.3f} pu'.format(e1q))
print('e1d0 = {:0.3f} pu'.format(e1d))
print('e_fd = {:0.3f} pu'.format(struct[i]['e_fd']))
print('P = {:0.4f} pu'.format(struct[i]['P']))
print('Q = {:0.4f} pu'.format(struct[i]['Q']))

print('delta_0 = {:0.3f} rad / {:0.3f} deg'.format(delta_0,np.rad2deg(delta_0)))


p_e = 0.80249029 pu
i_d = 0.830 pu
i_q = 0.376 pu
delta = 1.141 rad / 65.391 deg
omega = 1.000 pu
e1q = 0.924 pu
e1d = 0.417 pu
e1q0 = 0.924 pu
e1d0 = 0.417 pu
e_fd = 2.177 pu
P = 0.8000 pu
Q = 0.4360 pu
delta_0 = 1.141 rad / 65.391 deg


/home/jmmauricio/bin/anaconda3/lib/python3.6/site-packages/scipy/optimize/nonlin.py:474: RuntimeWarning: invalid value encountered in double_scalars
  and dx_norm/self.x_rtol <= x_norm))
/home/jmmauricio/bin/anaconda3/lib/python3.6/site-packages/scipy/optimize/nonlin.py:1038: RuntimeWarning: invalid value encountered in true_divide
  d = v / df_norm**2


In [227]:
def sm_ord4_pf(struct,i=0):
    x_idx = 0
    y_idx = 0
    
    H = struct[i]['H']
    X_d = struct[i]['X_d']
    X_q = struct[i]['X_q']
    X1d = struct[i]['X1d']
    X1q = struct[i]['X1q']
    X2d = struct[i]['X2d']
    X2q = struct[i]['X2q']
    X_l = struct[i]['X_l']
    R_s = struct[i]['R_s']
    D = struct[i]['D']
    Omega_b = struct[i]['Omega_b']
    T1d0 = struct[i]['T1d0']
    T1q0 = struct[i]['T1q0']
    T2d0 = struct[i]['T2d0']
    T2q0 = struct[i]['T2q0']
    V_m = struct[i]['V_m']
    theta = struct[i]['theta']
    delta = struct[i]['delta']
    e_fd = struct[i]['e_fd']
    p_m = struct[i]['p_m']
    p_e = struct[i]['p_e']
    i_d = struct[i]['i_d']
    i_q = struct[i]['i_q']
    omega = struct[i]['omega']
    x = struct[i]['x']
    f = struct[i]['f']
    y = struct[i]['y']
    g = struct[i]['g']

    delta =  struct[i]['x'][x_idx+0,0]%(2*np.pi)
    omega  = struct[i]['x'][x_idx+1,0]

    
    #i_d =   struct[i]['y'][y_idx+0,0]
    #i_q =   struct[i]['y'][y_idx+1,0]
   
    pi = np.pi
    sin = np.sin
    cos = np.cos 
    
    v_d = V_m*sin(delta-theta)
    v_q = V_m*cos(delta-theta)  

    i_d = (R_s*(e1d - v_d) + (X1q - X_l)*(e1q - v_q))/(R_s**2 + (X1d - X_l)*(X1q - X_l))
    i_q = (R_s*(e1q - v_q) - (X1d - X_l)*(e1d - v_d))/(R_s**2 + (X1d - X_l)*(X1q - X_l))

    e1q = - (X_d-X1d)*i_d + e_fd
    e1d =   (X_q-X1q)*i_q
    
    p_e = (v_q + R_s * i_q) * i_q + (v_d + R_s*i_d) * i_d
    
    ddelta = Omega_b * (omega-1)
    domega =10.0* (p_m - p_e - D*(omega-1))


    eq_id = v_q + R_s*i_q - e1q + (X1d - X_l)*i_d
    eq_iq = v_d + R_s*i_d - e1d - (X1q - X_l)*i_q
    
    P = v_d*i_d + v_q*i_q
    Q = v_q*i_d - v_d*i_q
   
    struct[i]['f'][x_idx+0,0] = float(ddelta)
    struct[i]['f'][x_idx+1,0] = float(domega)
    
    #struct[i]['g'][y_idx+0,0] = float(eq_id)
    #struct[i]['g'][y_idx+1,0] = float(eq_iq)

    struct[i]['p_e'] = float(p_e)
    struct[i]['i_d'] = float(i_d)
    struct[i]['i_q'] = float(i_q)
    struct[i]['delta'] = float(delta)
    struct[i]['omega'] = float(omega)
    struct[i]['e1q'] = float(e1q)
    struct[i]['e1d'] = float(e1d)
    struct[i]['P'] = float(P)
    struct[i]['Q'] = float(Q)
    return 0

struct[i]['D'] = 20
Dt = 0.001
struct[i]['x'][0] = 0.6
struct[i]['x'][1] = 1
struct[i]['p_m'] = 0.9
struct[i]['V_m'] = 1.0
struct[i]['theta'] = 0.0
struct[i]['e_fd'] = 2.8
N = 100
X = np.zeros((N,1))
for it in range(N):
    i = 0
    sm_ord4_pf(struct,i=0)    
    struct[i]['x'] += Dt* struct[i]['f']  
    X[it,0] = struct[i]['x'][0]

print('p_e = {:0.8f} pu'.format(struct[i]['p_e']))
print('i_d = {:0.3f} pu'.format(struct[i]['i_d']))
print('i_q = {:0.3f} pu'.format(struct[i]['i_q']))
print('delta = {:0.3f} rad / {:0.3f} deg'.format(struct[i]['delta'],np.rad2deg(struct[i]['delta'])))
print('omega = {:0.3f} pu'.format(struct[i]['omega']))
print('e1q = {:0.3f} pu'.format(struct[i]['e1q']))
print('e1d = {:0.3f} pu'.format(struct[i]['e1d']))
print('e1q0 = {:0.3f} pu'.format(e1q))
print('e1d0 = {:0.3f} pu'.format(e1d))
print('e_fd = {:0.3f} pu'.format(struct[i]['e_fd']))
print('P = {:0.4f} pu'.format(struct[i]['P']))
print('Q = {:0.4f} pu'.format(struct[i]['Q']))

print('delta_0 = {:0.3f} rad / {:0.3f} deg'.format(delta_0,np.rad2deg(delta_0)))

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 6), sharex = True)

axes.plot(X,'.')

UnboundLocalError: local variable 'e1d' referenced before assignment

In [222]:
import numpy as np
import matplotlib.pyplot as plt
#plt.style.use('presentation') # copy simlink from this folder
%matplotlib notebook
#colors_cycle=plt.rcParams.get('axes.prop_cycle')
#colors = [item['color'] for item in colors_cycle]


In [223]:
P_pu = struct[i]['P']
Q_pu = struct[i]['Q']
S = P_pu - 1j*Q_pu
V =  struct[i]['V_m']*np.exp(1j*struct[i]['theta'])
I = S/np.conj(V)
R_s = struct[i]['R_s']
X_d = struct[i]['X_d']
X_q = struct[i]['X_q']
X_l = struct[i]['X_l']
E = V + (R_s + 1j*(X_q+X_l))*I
delta = np.angle(E)
print('delta = {:0.3f} rad'.format(delta))

delta = 0.584 rad


In [189]:
E

(2.4627954431299854+1.7129048976075125j)

In [ ]:
d =dict(
H = 0.7,
X_d = 2.06,
X_q = 2.5,
X1d = 0.398,
X1q = 0.3,
X2d = 0.254,
X2q = 0.254,
X_l = 0.1,
R_s = 0.05,
D = 0.01,
Omega_b = 2*np.pi*50,
T1d0 = 7.8,
T1q0 = 3.0,
T2d0 = 0.066,
T2q0 = 0.075,
V_m = 1.0,
theta= 0.0,
delta = 0.2,
omega = 1.0,
e_fd = 1.1,
p_m = 1.0,
p_e = 0.0,
i_d = 0.0,
i_q = 0.0,
x=np.zeros((4,1)),
f=np.zeros((4,1)),
y=np.zeros((2,1)),
g=np.zeros((2,1))
)

d =dict(
S_n = 2220.0e6,
U_n = 24e3,
V_dc_n = 5e3,
H = 3.5,
X_d = 1.81,
X_q = 1.76,
X1d = 0.3,
X1q = 0.65,
X2d = 0.23,
X2q = 0.25,
X_l = 0.15,
R_s = 0.003,
D = 0.01,
Omega_b = 2*np.pi*50,
T1d0 = 8.0,
T1q0 = 1.0,
T2d0 = 0.03,
T2q0 = 0.07,
V_m = 1.0,
theta= 0.0,
delta = 0.2,
omega = 1.0,
e1q = 0.2,
e1d = 1.0,
e_fd = 1.1,
p_m = 1.0,
p_e = 0.0,
P = 1.0,
Q = 0.0,
P_pu = 1.0,
Q_pu = 0.0,
i_d = 0.0,
i_q = 0.0,
I_m = 0.0,
theta_i = 0.0,
x=np.zeros((4,1)),
f=np.zeros((4,1)),
y=np.zeros((2,1)),
g=np.zeros((2,1))
)


struct = d2np(d)

for item in d:
    print("    {:s} = struct[i]['{:s}']".format(item,item) )

In [ ]:
def sm_ord4_si(t,mode,struct,params_pf,params_simu):
    N = len(struct) # total number of bess_vsc_feeder
    
    params = struct
    
    for it in range(N):
        ix_0 = struct[i].ix_0
        nodes = struct[i].bus_nodes
        N_conductors = struct[i].N_conductors
        v_abcn = struct[0].V_node[nodes,:]
        i_abcn = struct[i].i_abcn
        gfeed_idx = struct[i].gfeed_idx    
        ctrl_mode = struct[i].ctrl_mode 
        
        S_n = struct[i]['S_n']
        U_n = struct[i]['U_n']
        V_dc_n = struct[i]['V_dc_n']
        H = struct[i]['H']
        X_d = struct[i]['X_d']
        X_q = struct[i]['X_q']
        X1d = struct[i]['X1d']
        X1q = struct[i]['X1q']
        X2d = struct[i]['X2d']
        X2q = struct[i]['X2q']
        X_l = struct[i]['X_l']
        R_s = struct[i]['R_s']
        D = struct[i]['D']
        Omega_b = struct[i]['Omega_b']
        T1d0 = struct[i]['T1d0']
        T1q0 = struct[i]['T1q0']
        T2d0 = struct[i]['T2d0']
        T2q0 = struct[i]['T2q0']
        V_m = struct[i]['V_m']
        theta = struct[i]['theta']
        delta = struct[i]['delta']
        omega = struct[i]['omega']
        e1q = struct[i]['e1q']
        e1d = struct[i]['e1d']
        e_fd = struct[i]['e_fd']
        p_m = struct[i]['p_m']
        p_e = struct[i]['p_e']
        P = struct[i]['P']
        Q = struct[i]['Q']
        P_pu = struct[i]['P_pu']
        Q_pu = struct[i]['Q_pu']
        i_d = struct[i]['i_d']
        i_q = struct[i]['i_q']
        I_m = struct[i]['I_m']
        theta_i = struct[i]['theta_i']
        x = struct[i]['x']
        f = struct[i]['f']
        y = struct[i]['y']
        g = struct[i]['g']

        S_b = S_n
        V_b = U_n/np.sqrt(3)
        I_b = S_b/(np.sqrt(3)*U_n)
        V_dc_b = V_dc_n
        p_m_pu = p_m/S_b
        V_m_pu = V_m/V_b

        alpha = np.exp(2.0/3*np.pi*1j)
        A_0a =  np.array([[1, 1, 1],
                          [1, alpha**2, alpha],
                          [1, alpha, alpha**2]])

        A_a0 = 1/3* np.array([[1, 1, 1],
                              [1, alpha, alpha**2],
                              [1, alpha**2, alpha]])
            
# %% initialization    
        if mode == 0:  # ini
            i_abcn_0 = np.zeros((4,1), dtype=np.complex128)

            S_ref = params_pf[0].gfeed_powers[gfeed_idx]            
            I_ref = params_pf[0].gfeed_currents[gfeed_idx]*np.exp(1j*np.angle(v_abcn[:,0]))

            I_abc_0 =  I_ref + np.conjugate(S_ref/v_abcn[:,0])
            I_n = -np.sum(I_abc_0)       
            V_abc_0 = v_abcn[0:4,0] 
            
            I_012 = A_a0 @ I_abc_0
            V_012 = A_a0 @ V_abc_0
            
            I_pos = I_012[1,0] 
            V_pos = V_012[1,0] 


            S_0 = v_abcn_0*np.conj(i_abcn_0)

            % powers and rotor angles
DAE.y(a.p) = a.u.*Bus.Pg(a.bus).*a.con(:,22);
DAE.y(a.q) = a.u.*Bus.Qg(a.bus).*a.con(:,23);
a.Pg0 = DAE.y(a.p);
Vg = DAE.y(a.vbus);
ag = DAE.y(a.bus);
V =  Vg.*exp(jay*ag);
S = DAE.y(a.p) - jay*DAE.y(a.q);
I = S./conj(V);
delta = angle(V + (ra + jay*(xq+xl)).*I);
DAE.x(a.delta) = a.u.*delta;

% d and q-axis voltages and currents
Vdq = a.u.*V.*exp(-jay*(delta-pi/2));
Idq = a.u.*I.*exp(-jay*(delta-pi/2));
vd = real(Vdq);
vq = imag(Vdq);
a.Id = real(Idq);
a.Iq = imag(Idq);

            params[it].v_abcn_0[:] = v_abcn_0
            params[it].i_abcn_0[:] = i_abcn_0     
            params[it].i_abcn[:] = np.copy(i_abcn_0)
            params[it].S_0[:] = S_0

            params_simu[0].x[ix_0+0,0] = params[it].soc_0


        delta =  struct[i]['x'][x_idx+0,0] %(2*np.pi)
        omega  = struct[i]['x'][x_idx+1,0]
        e1q =    struct[i]['x'][x_idx+2,0]
        e1d =    struct[i]['x'][x_idx+3,0]

        i_d =   struct[i]['y'][y_idx+0,0]
        i_q =   struct[i]['y'][y_idx+1,0]

        pi = np.pi
        sin = np.sin
        cos = np.cos 

        v_d = V_m_pu*sin(delta-theta)
        v_q = V_m_pu*cos(delta-theta)  

        p_e = (v_q + R_s * i_q) * i_q + (v_d + R_s*i_d) * i_d

        ddelta = Omega_b * (omega-1)
        domega = 1.0/(2*H)*(p_m_pu - p_e - D*(omega-1))
        de1q = 1/T1d0*(-e1q - (X_d-X1d)*i_d + e_fd)
        de1d = 1/T1q0*(-e1d + (X_q-X1q)*i_q)

        eq_id = v_q + R_s*i_q - e1q + (X1d - X_l)*i_d
        eq_iq = v_d + R_s*i_d - e1d - (X1q - X_l)*i_q

        P_pu = v_d*i_d + v_q*i_q
        Q_pu = v_q*i_d - v_d*i_q

        P = S_b * P_pu
        Q = S_b * Q_pu

        I_re = i_d * sin(delta) + i_q * cos(delta)
        I_im = i_d * cos(delta) - i_q * sin(delta)  

        I = I_b*(I_re + 1j*I_im)
        I_m = np.abs(I)
        theta_i = np.angle(I)

        struct[i]['f'][x_idx+0,0] = float(ddelta)
        struct[i]['f'][x_idx+1,0] = float(domega)
        struct[i]['f'][x_idx+2,0] = float(de1q)
        struct[i]['f'][x_idx+3,0] = float(de1d)

        struct[i]['g'][y_idx+0,0] = float(eq_id)
        struct[i]['g'][y_idx+1,0] = float(eq_iq)

        struct[i]['p_e'] = float(p_e)
        struct[i]['i_d'] = float(i_d)
        struct[i]['i_q'] = float(i_q)
        struct[i]['delta'] = float(delta)
        struct[i]['omega'] = float(omega)
        struct[i]['e1q'] = float(e1q)
        struct[i]['e1d'] = float(e1d)
        struct[i]['P_pu'] = float(P_pu)
        struct[i]['Q_pu'] = float(Q_pu)
        struct[i]['P'] = float(P)
        struct[i]['Q'] = float(Q)
        struct[i]['I_m'] = float(I_m)
        struct[i]['theta_i'] = float(theta_i)
        return 0

#class problem(object)
def problem(lam, i=0):
    #print(lam.shape)
    struct[i]['x'] = lam[0:4,:]
    struct[i]['y'] = lam[4:6,:]
    Phi = np.zeros((6,1))
    sm_ord4_si(struct,i=0)
    Phi[0:4,:] = struct[i]['f']
    Phi[4:6,:] = struct[i]['g']   
    return Phi
    
sm_ord6(struct,i=0)
i=0
lam_0 = np.ones((6,1))
p_m_0 = 0.9
lam_0[0] = 1.2
lam_0[1] = 1.0
lam_0[2] = 1.0
struct[i]['D'] = 0.0
struct[i]['R_s'] = 0.003
struct[i]['p_m'] = 0.903*2220e6
struct[i]['e_fd'] = 2.282*0.7
struct[i]['V_m'] = 24e3/np.sqrt(3) # Volt
struct[i]['theta'] = np.deg2rad(0.0)
solution = sopt.broyden1(problem,lam_0,  maxiter=1000)
solution
P = struct[i]['P']
Q = struct[i]['Q']
S = P + 1j*Q
phi = np.arccos(P/np.abs(S))
print('p_e = {:0.8} pu'.format(struct[i]['p_e']))
print('i_d = {:0.3} pu'.format(struct[i]['i_d']))
print('i_q = {:0.3} pu'.format(struct[i]['i_q']))
print('delta = {:0.3} rad / {:0.3} deg'.format(struct[i]['delta'],np.rad2deg(struct[i]['delta'])))
print('omega = {:0.3} pu'.format(struct[i]['omega']))
print('e1q = {:0.3} pu'.format(struct[i]['e1q']))
print('e1d = {:0.3} pu'.format(struct[i]['e1d']))
print('P = {:6.3f} MW'.format(struct[i]['P']/1e6))
print('Q = {:0.3f} Mvar'.format(struct[i]['Q']/1e6))
print('I_pos = {:6.3f} | {:0.3f}º kA'.format(struct[i]['I_m']/1e3,np.rad2deg(struct[i]['theta_i'])))
print('phi = {:6.3f} º'.format(np.rad2deg(phi)))


In [ ]:
struct = d2np(d)
struct['v_ds'] = 325.0
struct['v_qs'] = 0.0
struct['omega_r'] = Omega_b*1.0
Dt = 1.0e-4
struct[0]['x']= np.zeros((6,1))
struct[0]['x'][4,0] =   Omega_b*1.0
Tau_e = []
T =[]
N_steps = 100000
X = np.zeros((N_steps,6))

def f_eval(struct):
    dfim(struct,0)
    wecs_mech_1(struct,0)
    return struct[0]['f']
    
    
for it in range(N_steps):
    t = Dt*it
    if t>1.0:
        struct[0]['tau_t'] = 1.0e4
    dfim(struct,0)
    f1 = np.copy(f_eval(struct))
    x1 = np.copy(struct[0]['x'])
    struct[0]['x'] = np.copy(x1 + Dt*f1)
    dfim(struct,0)
    struct[0]['x'] = np.copy(x1 + 0.5*Dt*(f1 + f_eval(struct)))
    struct[0]['tau_r'] =  -struct[0]['tau_e']
    Tau_e += [float(struct['tau_e'])]
    X[it,:] = np.copy(struct[0]['x'][:]).T
    T +=[t]    
    

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), sharex = True)
axes.plot(T,X[:,0])
axes.plot(T,X[:,1])
axes.plot(T,X[:,2])
axes.plot(T,X[:,3])

fig.savefig('dfim_tau_e.svg', bbox_inches='tight')


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), sharex = True)
axes.plot(T,Tau_e)

fig.savefig('dfim_tau_e.svg', bbox_inches='tight')


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 5), sharex = True)
axes.plot(T,X[:,4])

In [ ]:
X[:,4]

In [10]:
20*15.68

313.6